## Load US CENSUS All-Sector Report data from 2017 datasets (see https://data.census.gov/cedsci/ and set your own parameters) for original sources

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : https://redhat-osc-physical-landing-647521352890.s3.amazonaws.com/US_CENSUS/US_Census-All-Sector-Report-2017/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

## Based on OSC Ingest Tutorial by Erik Erlandson (GitHub: erikerlandson)

## python dependencies
The following are python packages commonly used for data ingest.
If your jupyter environment does not already have these,
you can copy and paste these into a live code cell to install them.

Run these in a notebook cell if you need to install onto your nb env
```
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

## Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [ ]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

## S3 and boto3

In [ ]:
import boto3
s3_src = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
src_bucket = s3_src.Bucket(os.environ['S3_LANDING_BUCKET'])

s3_trino = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)
trino_bucket = s3_trino.Bucket(os.environ["S3_DEV_BUCKET"])

## Connecting to Trino with sqlalchemy

In [ ]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev',
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

## Load Data Dictionary for US Census 2017 All-sector report

In [ ]:
import pandas as pd

data_dictionary = 'ECNCOMP2017.EC1700COMP_metadata_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{data_dictionary}')
bObj.download_file(f'/tmp/ingest-{data_dictionary}')
dd_df = pd.read_csv(f'/tmp/ingest-{data_dictionary}', sep=',', header=None, engine='c')
dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{data_dictionary}')
dd_df

The US Census maintains a list of SIC codes, but it's very empty compared with the SEC version.  It does give 2-digit code definitions, however.  But they are not very complete, either.

In [ ]:
survey_report = 'ECNCOMP2017.EC1700COMP_data_with_overlays_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{survey_report}')
bObj.download_file(f'/tmp/ingest-{survey_report}')
df = pd.read_csv(f'/tmp/ingest-{survey_report}', sep=',', header=0, engine='c')
os.unlink(f'/tmp/ingest-{survey_report}')

df = df.drop(df.index[0])
df.reset_index (drop=True, inplace=True)
df

## `osc-ingest-tools`

## Enforcing valid SQL column names

In [ ]:
import osc_ingest_trino as osc
from io import BytesIO

osc.enforce_sql_column_names(df, inplace=True)
df

## Trino catalog, schema, table

In [ ]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'us_census'
ingest_table = 'all_sector_survey_2017'
dd_table = 'data_dictionary'
sic4_table = 'sic4'
# sic2_table = 'sic2'
sic_ranges_table = 'sic_ranges'

## Staging parquet files for SURVEY REPORT trino table

In [ ]:
# DF was loaded from the SURVEY_REPORT file

buf = BytesIO()
df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{ingest_table}/data.parquet')

## Declaring a trino table on top of a raw parquet file

In [ ]:
qres = engine.execute(f'create schema if not exists {ingest_schema}')
print(qres.fetchall())

qres = engine.execute(f'drop table {ingest_schema}.{ingest_table}')

columnschema = osc.create_table_schema_pairs(df)

tabledef = f"""
create table {ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{ingest_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

Now do the same for the data dictionary in DD_DF

In [ ]:
buf = BytesIO()
dd_df.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{dd_table}/data.parquet')

In [ ]:
qres = engine.execute(f'drop table {ingest_schema}.{dd_table}')

columnschema = osc.create_table_schema_pairs(dd_df)

tabledef = f"""
create table if not exists {ingest_schema}.{dd_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{dd_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

Load SIC4 codes from XML files (somewhat hidden) at SEC.GOV (https://www.sec.gov/xbrl/site/sic/2020/sic-lab-2020-01-31.xml)

In [ ]:
# %pip install xmltodict

import xmltodict

with open('sic-lab-2020-01-31.xml') as fd:
    doc = xmltodict.parse(fd.read())

sic4_xml = [x['#text'] for x in doc['link:linkbase']['link:labelLink']['link:label']]
title = sic4_xml[0]
df = pd.DataFrame(data=sic4_xml[1:], columns=['row'])
range_mask = df.row.str.contains(' to ')
df_ranges = df.loc[range_mask]
df_defs = df.loc[~range_mask]
df_ranges.info(verbose=True)
df_defs.info(verbose=True)

In [ ]:
def sic_range_tuple(x):
    l = x.row.split(' ', 3)
    r = l[3].rsplit(' ',1)
    return '-'.join([l[0], l[2]]), l[0], l[2], r[0]

df_ranges[['range', 'l', 'r', 'range_desc']] = df_ranges.apply(sic_range_tuple, axis=1, result_type='expand')
df_ranges.l = df_ranges.l.astype('int32')
df_ranges.r = df_ranges.r.astype('int32')
df_ranges.drop(columns=['row'],inplace=True)
df_ranges.reset_index (drop=True, inplace=True)
df_ranges

In [ ]:
def sic_tuple(x):
    l = x.row.split(' ', 1)
    r = l[1].rsplit(' ',1)
    return l[0], r[0]

df_defs[['sic', 'desc']] = df_defs.apply(sic_tuple, axis=1, result_type='expand')
df_defs.sic = df_defs.sic.astype('int32')
df_defs.drop(columns=['row'],inplace=True)
df_defs.reset_index (drop=True, inplace=True)
df_defs

In [ ]:
buf = BytesIO()
df_defs.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{sic4_table}/data.parquet')

In [ ]:
qres = engine.execute(f'drop table {ingest_schema}.{sic4_table}')

columnschema = osc.create_table_schema_pairs(df_defs)

tabledef = f"""
create table if not exists {ingest_schema}.{sic4_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{sic4_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

In [ ]:
buf = BytesIO()
df_ranges.to_parquet(path=buf, index=False)
buf.seek(0)

trino_bucket.upload_fileobj(Fileobj=buf,
                            Key=f'trino/{ingest_schema}/{sic_ranges_table}/data.parquet')

In [ ]:
qres = engine.execute(f'drop table {ingest_schema}.{sic_ranges_table}')

columnschema = osc.create_table_schema_pairs(df_ranges)

tabledef = f"""
create table if not exists {ingest_schema}.{sic_ranges_table}(
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{sic_ranges_table}/'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

## SQL queries on our new table

In [ ]:
sql = f"""
select * from {ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine).convert_dtypes()

We don't use the obsolete/limited SIC2 and SIC4 data from US Census